**WORK FLOW:**


SECTIONS FOR PRESENTATION
        
        IMPORTS
        MY DEFINED FUNCTIONS
        COLUMN DESCRIPTIONS
        MODEL PREP
        LR MODEL

**IMPORTS**

In [1]:
import joblib
from dask_ml.model_selection import train_test_split
from sklearn.model_selection import cross_validate, GridSearchCV

In [2]:

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from statsmodels.stats.diagnostic import het_breuschpagan
from sklearn import tree, ensemble
import pandas as pd
import numpy as np
from sklearn.svm import SVR, SVC
from scipy import stats
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_regression, f_classif
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import math
from sklearn.metrics import mean_absolute_error,confusion_matrix, classification_report
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.eval_measures import mse, rmse
import warnings
import statsmodels.formula.api as smf
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
%matplotlib inline

import dask.bag as db
import dask.dataframe as dd

from dask_ml.model_selection import train_test_split
from dask_ml.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings("ignore")

from dask.distributed import Client, progress

client = Client(n_workers=4, threads_per_worker=2, memory_limit='2GB')
client

Client Scheduler: tcp://127.0.0.1:58458 Dashboard: http://127.0.0.1:58457/status,Cluster Workers: 4 Cores: 8 Memory: 8.00 GB


**MY DEFINED FUNCTIONS**

In [3]:
def print_vif(x):
    """Utility for checking multicollinearity assumption
    
    :param x: input features to check using VIF. This is assumed to be a pandas.DataFrame
    :return: nothing is returned the VIFs are printed as a pandas series
    """
    # Silence numpy FutureWarning about .ptp
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        x = sm.add_constant(x)

    vifs = []
    for i in range(x.shape[1]):
        vif = variance_inflation_factor(x.values, i)
        vifs.append(vif)

    print("VIF results\n-------------------------------")
    print(pd.Series(vifs, index=x.columns))
    print("-------------------------------\n")
    
def eval_preds(y_true, y_pred):
    error = y_true - y_pred

    rmse = np.sqrt((error ** 2).mean())
    mae = error.abs().mean()
    mape = (error / y_true).abs().mean()

    print(f"rmse {rmse:.2f}")
    print(f"mae {mae:.2f}")
    print(f"mape {mape:.2f}")

    line_pts = [y_true.min(), y_true.max()]
    plt.scatter(y_true, y_pred)
    plt.plot(line_pts, line_pts, c="red", ls="--", alpha=0.5)
    plt.xlabel("Actual")
    plt.ylabel("Fit")
    plt.show()
    
def one_hot_encode(X, encode_cols, index=None):
    other_cols = [c for c in X.columns if c not in encode_cols]

    ct = ColumnTransformer(
        #   Format
        #   [("name of step", what_to_do(), [what columns to do it to])]
        [("one hot encode", OneHotEncoder(drop="first", sparse=False), encode_cols)],
        remainder="passthrough",
    )

    ct.fit(X)

    # This is not the flexible and definitely not the most
    # readable way to get column names, a function would be better
    encoded_names = ct.transformers_[0][1].get_feature_names()
    encoded_names = list(encoded_names)

    X_encoded = ct.transform(X)
    X_encoded = pd.DataFrame(X_encoded, columns=encoded_names + other_cols, index=index)

    return X_encoded

**Column descriptions** 'G', 'W', 'BARTHAG', 'TOR', 'ORB', 'DRB', 'FTR', '2P_O', '2P_D',
       '3P_D'

RK: The ranking of the team at the end of the regular season according to barttorvik

TEAM:The Division I college basketball school

CONF:The Athletic Conference in which the school participates in (A10 = Atlantic 10, ACC = Atlantic Coast Conference, AE = America East, Amer = American, ASun = ASUN, B10 = Big Ten, B12 = Big 12, BE = Big East, BSky = Big Sky, BSth = Big South, BW = Big West, CAA = Colonial Athletic Association, CUSA = Conference USA, Horz = Horizon League, Ivy = Ivy League, MAAC = Metro Atlantic Athletic Conference, MAC = Mid-American Conference, MEAC = Mid-Eastern Athletic Conference, MVC = Missouri Valley Conference, MWC = Mountain West, NEC = Northeast Conference, OVC = Ohio Valley Conference, P12 = Pac-12, Pat = Patriot League, SB = Sun Belt, SC = Southern Conference, SEC = South Eastern Conference, Slnd = Southland Conference, Sum = Summit League, SWAC = Southwestern Athletic Conference, WAC = Western Athletic Conference, WCC = West Coast Conference)

GNumber: of games played

WNumber: of games won

ADJOE: Adjusted Offensive Efficiency (An estimate of the offensive efficiency (points scored per 100 possessions) a team would have against the average Division I defense)

ADJDE: Adjusted Defensive Efficiency (An estimate of the defensive efficiency (points allowed per 100 possessions) a team would have against the average Division I offense)

BARTHAG: Power Rating (Chance of beating an average Division I team)

EFG_O: Effective Field Goal Percentage Shot

EFG_D: Effective Field Goal Percentage Allowed

TOR: Turnover Percentage Allowed (Turnover Rate)

TORD: Turnover Percentage Committed (Steal Rate)

ORB: Offensive Rebound Percentage

DRB: Defensive Rebound Percentage

FTR: Free Throw Rate (How often the given team shoots Free Throws)

FTRD: Free Throw Rate Allowed

2P_O: Two-Point Shooting Percentage

2P_D: Two-Point Shooting Percentage Allowed

3P_O: Three-Point Shooting Percentage

3P_D: Three-Point Shooting Percentage Allowed

ADJ_T: Adjusted Tempo (An estimate of the tempo (possessions per 40 minutes) a team would have against the team that wants to play at an average Division I tempo)

WAB: Wins Above Bubble (The bubble refers to the cut off between making the NCAA March Madness Tournament and not making it)

POSTSEASON: Round where the given team was eliminated or where their season ended (R68 = First Four, R64 = Round of 64, R32 = Round of 32, S16 = Sweet Sixteen, E8 = Elite Eight, F4 = Final Four, 2ND = Runner-up, Champion = Winner of the NCAA March Madness Tournament for that given year)

SEED: Seed in the NCAA March Madness Tournament YEAR: Season

In [4]:
cbb = dd.read_csv('cbb.csv')



In [5]:
print(cbb)

Dask DataFrame Structure:
                 TEAM    CONF      G      W    ADJOE    ADJDE  BARTHAG    EFG_O    EFG_D      TOR     TORD      ORB      DRB      FTR     FTRD     2P_O     2P_D     3P_O     3P_D    ADJ_T      WAB POSTSEASON     SEED   YEAR
npartitions=1                                                                                                                                                                                                                  
               object  object  int64  int64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64     object  float64  int64
                  ...     ...    ...    ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...        ...      ...    ...
Dask Name: read-csv, 1 tasks


-------------------------------------------------------------------------------------------------------------------------------


**MODEL PREP** (for more  indepth reasons of why i maniplulated the data the way i did see EDA section)

In [6]:
#i noticed a huge difference in number of games and did some double checking the 'G' values 24-26 are missing games. 
#I also noticed the 'CONF' value Slnd has a lot off errrors. CONF == Ind only has one value.
#so i am droppnig rows with (conf == Slnd or Ind) and rows G <=26
cbb = cbb.loc[cbb['G'] >= 27]
cbb = cbb.loc[cbb['CONF'] != 'Slnd']
cbb = cbb.loc[cbb['CONF'] != 'Ind']


In [7]:
# trying to predict who would make NCAA basketball tourn using'SEED' as prediction 'POSTSEASON' and 'WAB' are cheat 
#columns so dropping them
cbb_seed_con = cbb.copy()
cbb_seed_con = cbb_seed_con.drop(columns=['POSTSEASON', 'WAB'])

In [8]:
#after looking at correlation  and vif decided to drop the following columns  #'ADJOE', 'ADJDE',
cbb_seed_con =cbb_seed_con.drop(columns=[ 'CONF','EFG_O', 'EFG_D', 'ADJ_T', '3P_O', 'FTRD','TORD','ADJOE', 'ADJDE'])
cbb_seed_con =cbb_seed_con.drop(columns=['TEAM', 'YEAR'])
cbb_seed_con.head()
                  

,G,W,BARTHAG,TOR,ORB,DRB,FTR,2P_O,2P_D,3P_D,SEED
0,40,33,0.9531,15.4,40.7,30.0,32.3,53.9,44.6,36.2,1.0
1,40,36,0.9758,12.4,32.1,23.7,36.2,54.8,44.7,37.5,1.0
2,40,33,0.9375,14.0,25.5,24.9,30.7,54.7,46.8,33.2,3.0
3,38,31,0.9696,17.7,27.4,28.7,32.9,52.8,41.9,29.7,3.0
4,39,37,0.9728,16.2,30.0,26.2,39.0,56.3,40.0,29.0,1.0


In [9]:
print(cbb_seed_con)

Dask DataFrame Structure:
                   G      W  BARTHAG      TOR      ORB      DRB      FTR     2P_O     2P_D     3P_D     SEED
npartitions=1                                                                                               
               int64  int64  float64  float64  float64  float64  float64  float64  float64  float64  float64
                 ...    ...      ...      ...      ...      ...      ...      ...      ...      ...      ...
Dask Name: drop_by_shallow_copy, 13 tasks


In [10]:
cbb_seed_con['SEED'] = cbb_seed_con['SEED'].fillna(17)

In [11]:
cbb_seed_con.SEED.unique().compute()

0      1.0
1      3.0
2      2.0
3      4.0
4      6.0
5      9.0
6     11.0
7     10.0
8      7.0
9      5.0
10    17.0
11     8.0
12    16.0
13    12.0
14    13.0
15    14.0
16    15.0
Name: SEED, dtype: float64

**MODEL USED FOR 'SEED' PREDICTION:** ols

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [13]:
#define X and Y
y = cbb_seed_con['SEED']
X = cbb_seed_con.drop(columns=['SEED'])
#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 85)

In [14]:

lr = LogisticRegression()
with joblib.parallel_backend('dask'):
    scores = cross_validate(lr, X_train.compute(), y_train.compute(), cv=4)
    
scores

{'fit_time': array([0.56948924, 0.3430829 , 0.35804367, 0.5624969 ]),
 'score_time': array([0.00498557, 0.00997281, 0.0039897 , 0.00797844]),
 'test_score': array([0.79518072, 0.79819277, 0.79819277, 0.81873112])}